# Naive Bayes (the easy way)

We'll cheat by using sklearn.naive_bayes to train a spam classifier! Most of the code is just loading our training data into a pandas DataFrame that we can play with:

In [20]:
import os
import io
import numpy
import pandas as pd
from pandas import DataFrame
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

def readFiles(path):
    for root, dirnames, filenames in os.walk(path):
        for filename in filenames:
            path = os.path.join(root, filename)

            inBody = False
            lines = []
            f = io.open(path, 'r', encoding='latin1')
            for line in f:
                if inBody:
                    lines.append(line)
                elif line == '\n':
                    inBody = True
            f.close()
            message = '\n'.join(lines)
            yield path, message


def dataFrameFromDirectory(path, classification):
    rows = []
    index = []
    for filename, message in readFiles(path):
        rows.append({'message': message, 'class': classification})
        index.append(filename)

    return DataFrame(rows, index=index)

data = DataFrame({'message': [], 'class': []})

data = pd.concat([data, dataFrameFromDirectory("emails/spam", "spam")]);
data = pd.concat([data, dataFrameFromDirectory("emails/ham", "ham")])

#For Pandas 1.3:
#data = data.append(dataFrameFromDirectory('emails/spam', 'spam'))
#data = data.append(dataFrameFromDirectory('emails/ham', 'ham'))

Let's have a look at that DataFrame:

In [21]:
data.head()

,message,class
emails/spam/00281.db28f3aab77ff478279d8de20d572b42,\n\n<html><body><center>\n\n\n\n<table bgcolor...,spam
emails/spam/00057.0a2e17bde9485e999ac2259df38528e2,Lowest rates available for term life insurance...,spam
emails/spam/00105.00951a21b8464f4eb4e106d6b14c68b6,This is a multi-part message in MIME format.\n...,spam
emails/spam/00457.f8db516c753eff2c82cfb89b33bd2620,<HTML>\n\n<HEAD>\n\n</HEAD>\n\n<BODY>\n\nVIAGR...,spam
emails/spam/00111.ae6aba48f8aa83849be067076eea8ce5,Whiter teeth and a brighter smile are just a c...,spam


Now we will use a CountVectorizer to split up each message into its list of words, and throw that into a MultinomialNB classifier. Call fit() and we've got a trained spam filter ready to go! It's just that easy.

In [22]:
vectorizer = CountVectorizer()
counts = vectorizer.fit_transform(data['message'].values)

classifier = MultinomialNB()
targets = data['class'].values
classifier.fit(counts, targets)

MultinomialNB()

Let's try it out:

In [23]:
examples = ['Free Viagra now!!!', "Hi Bob, how about a game of golf tomorrow?"]
example_counts = vectorizer.transform(examples)
predictions = classifier.predict(example_counts)
predictions

array(['spam', 'ham'], dtype='<U4')

## Activity

Our data set is small, so our spam classifier isn't actually very good. Try running some different test emails through it and see if you get the results you expect.

If you really want to challenge yourself, try applying train/test to this spam classifier - see how well it can predict some subset of the ham and spam emails.

In [24]:
emails = ["Enlarge your pension", "Sexy girls", "You won't believe", 
         "Easy way to earn a million of dollars", "Job position at PWC", 
         "Order reciept", "Thank you for your feedback",
         "Easy passive income", "Top trading strategies"]

In [25]:
email_counts = vectorizer.transform(emails)

In [26]:
predictions = classifier.predict(email_counts)

In [27]:
predictions

array(['spam', 'spam', 'ham', 'spam', 'ham', 'spam', 'ham', 'ham', 'spam'],
      dtype='<U4')